In [ ]:
import json
import pandas as pd
import numpy as np
import os
os.chdir("../")
from utils import *
import re

import glob
import itertools
from PIL import Image
from fuzzywuzzy import fuzz

In [ ]:
def fuzzy_score(text, value):
    """
    Function for fuzzy matching given two strings

    Args:
        text (string): ocr data text
        value (string): entity label

    Returns:
        float : Fuzzy match score
    """

    return fuzz.ratio(text, value)

In [ ]:
source_folder = '../../test_data/'

gt_labels = glob.glob(f'{source_folder}test_labels/*labels.json')
predicted_labels = glob.glob(f'{source_folder}test_outputs/*.json')
images_test = [glob.glob(e) for e in [f'{source_folder}test_images/*.jpeg', f'{source_folder}test_images/*.PNG', f'{source_folder}test_images/*.png']]
images_test = list(itertools.chain(*images_test)) #flatten the list


idx = 0
filename_gt = gt_labels[idx]
filename_label = predicted_labels[idx]
test_img = images_test[idx]


with open(f"./{filename_gt}", 'r', encoding="utf8") as f:
    data_label = json.load(f)

with open(f"./{filename_label}", 'r', encoding="utf8") as f:
    data_predicted = json.load(f)


img = Image.open(test_img)
width, height = img.size


assert len(gt_labels) == len(predicted_labels) == len(images_test)


In [ ]:
df = parse_text_ocr_entities(data_predicted)

#CREATE CONSISTENCY FOR THE CATEGORIES DURING LABELLING
category_mapping_output = {"From": "Exporter Name",
"To": "Recipient Name",
}

df = df.replace({"level": category_mapping_output})
df = df.replace('\n',' ', regex=True) #clean up some messy strings

In [ ]:
base_data = data_label["labels"]

#Parse original output
page_list2, text_list2,bb_list2, level_cat = parse_text_labels(base_data)

# Format text into target format per page
df2 = pd.DataFrame([page_list2, text_list2,bb_list2, level_cat]).T
df2.columns = ["page", "text","bbox", "level"]
df_label = df2.copy()

# convert bounding boxes
df_label["bbox_formatted"] = [convert_inches_pixel_normalized_vector(list(df_label["bbox"].iloc[i]), pixel_conv_x=width, pixel_conv_y=height) for i in range(len(df_label))]


# Iterate by page
text_by_page_formatted_label = []
for i in df_label["page"].unique():
    df_label_page= df_label[df_label.page == i]
    text_by_page_formatted_label.append([format_json_sublevel_label(df_label_page, u) for u in range(len(df_label_page))])

In [ ]:
# #CREATE CONSISTENCY FOR THE CATEGORIES DURING LABELLING
# category_mapping = {""}

# df_label = df_label.replace({"level": category_mapping})

In [ ]:
combined_df = df.merge(df_label, how="outer", left_on="level", right_on="level", suffixes=["_predicted", "_ground_truth"]).drop(columns=["bbox_predicted", "bbox_ground_truth"])


In [ ]:
tt = combined_df[['text_ground_truth','level']].fillna(" ")
tt = tt.groupby(['level'], as_index=False).agg({'text_ground_truth': ' '.join})
tt = tt.merge(df, how="outer", left_on="level", right_on="level")
tt = tt.fillna(" ")
tt["word_similarity"] = [fuzzy_score(tt["text_ground_truth"][i], tt["text"][i]) for i in range(len(tt))]
tt